### 0.
Install pacakges:

```python
pip install aicsimageio
pip install cellpose
```

Download models:

Make sure you are running this notebook with access to a GPU. If you don't have access, set the parameter gpu=False (below), but be warned that running model inference will take much longer.

In [14]:
from confocalQuant.segmentation import toggle_filters, load_3D, int_to_float
from aicsimageio import AICSImage
from cellpose import models

### 1.
First, specify which channels to load from the image listed in path. You can specify as many channels as you want. Takes .czi files as input.

In [4]:
# specify channels to load
channels = [0,1,2] #### ADD CUSTOM CHANNELS ####

# specify path
path = './data/neuronbatch10132023_mitohealth/B06-1.czi' #### ADD CUSTOM PATH ####

### 2.
Visualize the image by toggling through the different parameters:

- *show*: select which channels you want to show. Hold shift to select multiple channels at once. A maximum of 3 channels can be selected at once. Channels will be shown as Red, Green, Blue in ascending order. To modify this, change the order in which you load the channels above. 
- *adjust*: toggle which images to adjust with sliders below
- *zi_slider*: toggle through images on the z-axis
- *gamma_slider*: apply gamma filter for visualization. 
- *median slider*: apply median filter to remove sensor noise
- *background slider*: subtract background signal from all values (in %).
- *lower_slider*: set values below this threshold to 0 (in %).
- *upper_slider*: set values above this threshold to max value (in %).

N.B. The first time you call the function toggle_filters will take longer than subsequent rounds, as some of the code gets compiled into machine code first, to substantially speed up the functions in subsequent calls.

In [12]:
# load image
img = AICSImage(path)
out = load_3D(img, channels)
out_float = int_to_float(out)

# visualize the image
toggle, median, background_dict, gamma_dict, upper_dict, lower_dict = toggle_filters(out_float)
toggle

### 3.
Choose parameters above that remove background signal (from tissue autofluorescence or contaminating cells), smooth over sensor noise if present, remove outlier signal and highlight the cells you care about segmenting. However, don't go overboard with thesholding, otherwise the model will have trouble. Only the channels you will actually be using in the model should be adjusted in this way (i.e. a cell body marker and, optionally, a nuclear marker).

In [ ]:
### MODIFY BASED ON SETTINGS ABOVE ###
kernel = 1 # (i.e. median filter value)
background_dict = {0: 0, 1: 95, 2: 0.0} # (i.e. for every channel, background value)
gamma_dict =  {0: 1, 1: 1, 2: 1} # (i.e. for every channel, the gamma value)
lower_dict = {0: 0, 1: 0, 2: 0} # (i.e. for every channel, the lower value)
upper_dict =  {0: 100, 1: 99.5, 2: 100} # (i.e. for every channel, the upper value)

In [ ]:
# apply the filters
out_med = run_med_filter(out_float, kernel = kernel)
out_float_subtract = bgrnd_subtract(out_med, np.array(list(background_dict.values())))
g_correct = gamma_correct_image(out_float_subtract,  gamma_dict, lower_dict, upper_dict)

### 4.

Perform model segmentation. Start with the default parameters, then go back to modify them once you've viewed initial results. 

In [ ]:
### MODEL PARAMS ###

diameter=17 # expected cell body diameter in um
channels=[2,0] # channels to use for segmentation. 1=red, 2=green, 3=blue; see the image above to determine what colors you care about. First position = cell body channel. Second position = nuclear channel. If you don't want to use a nuclear channel for segmentation, enter '0' in the second position.
min_size=500 # minimum number of expected points ("voxels") per cell

In [ ]:
# get anisotropy
anisotropy = get_anisotropy(img)
print('Anisotropy: ' + str(anisotropy))

# load model
model = models.Cellpose(gpu = True, model_type='cyto2')

# run inference
masks, flows = do_inference(g_correct, do_3D=True, model=model, anisotropy=anisotropy, diameter=diameter, channels=channels, channel_axis=3, z_axis=0, min_size=min_size, normalize = False)

### 5. 
Toggle through the segmentation results to evaluate them.

In [ ]:
# toggle through segmentation
extracted = extract_channels([0,1], g_correct) # enter which channels to show here (up to 3 channels)
toggle_segmentation(float_to_int(extracted), masks) # you can optionally pass out_float, out_med, out_float_subtract, or apply different filters to out_float to help with the visualization

### 6. 
You may want to repeat steps 1-5 to make sure the parameters you are choosing work well for different images.
Once you've found good parameters, modify the example.sbatch file to do inference on all your images.

```bash
sbatch example.sbatch
```

### 7. 

Load, explore, and plot quantifications of the results.

In [ ]:
from confocalQuant.segmentation import extract_sbatch_parameters, get_czi_files
from confocalQuant.plotting import return_results, concatenate_Y, add_metadata, exclude_files, modify_keep_files

In [ ]:
#### SPECIFY PATH TO SBATCH FILE ###
path_to_sbatch_file = 
path_to_czi_files = 
nuclear_col_idx = 
soma_col_idx = 
nuclear_percentile = 
soma_percentile = 
colnames = ['DAPI', 'NeuN', 'mito', 'mito-b', 'ID']
path_to_meta = 

In [ ]:
# load & concatenate the data
mat, masks, Y, Ncells, Nzi, cells_per_job = return_results(path_to_sbatch_file)
all_file_names = get_czi_files(path_to_czi_files)
data = concatenate_Y(all_file_names, cells_per_job, Ncells_per_job, nuclear_col_idx, soma_col_idx, nuclear_percentile, soma_percentile, colnames)
add_metadata(data, path_to_meta)

In [ ]:
# exclude files from quantification and segmentation toggling (optional)
keep_files = exclude_files(exclude, all_file_names)

# if there are none to remove, run
keep_files = list(range(len(all_file_names)))

In [ ]:
# toggle through all segmentations

In [ ]:
#### SPECIFY IDS TO REMOVE ####
ids_to_remove = []

In [ ]:
# remove IDs from keep_files if you think it's necessary based on poor segmentation or image quality (otherwise re-run with differnet parameters)
keep_files = modify_keep_files(keep_files, ids_to_remove)

In [ ]:
# modify data to only include the keep_files IDs
data = data[[x in keep_files for x in data['ID']]]

In [ ]:
# plot effects
plot_effect(data, 'condition', 'G2 vehicle', 'G2 CDP-choline', 'mito-b')

In [ ]:
# plot and save representative image panels